In [46]:
import warnings
warnings.filterwarnings("ignore")

In [47]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
from pyspark.sql import functions as F

In [48]:
spark= SparkSession.builder\
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1")\
    .config("spark.jars", "sqljdbc42.jar")\
    .getOrCreate()

24/09/23 18:25:37 INFO MicroBatchExecution: Streaming query has been idle and waiting for new data more than 10000 ms.


In [49]:
spark

In [50]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "moviesProject") \
    .option("enable.auto.commit", True) \
    .option("startingOffsets", "earliest") \
    .load()

In [51]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [52]:
schema = StructType([
    StructField("show_id", StringType(), True),
    StructField("type", StringType(), True),
    StructField("title", StringType(), True),
    StructField("director", StringType(), True),
    StructField("cast", StringType(), True),
    StructField("country", StringType(), True),
    StructField("date_added", StringType(), True),
    StructField("release_year", StringType(), True),
    StructField("rating", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("listed_in", StringType(), True),
    StructField("description", StringType(), True),
    StructField("source", StringType(), True),
    StructField("total_views", StringType(), True),
    StructField("user_reviews", StringType(), True),
    StructField("user_rating", StringType(), True)
])

In [53]:
json_df = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

In [54]:
json_df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: string (nullable = true)
 |-- user_reviews: string (nullable = true)
 |-- user_rating: string (nullable = true)



In [55]:
Updateemptystring = json_df.replace("", None)


In [56]:
removewhitespace = Updateemptystring.select([ltrim(c).alias(c) for c in Updateemptystring.columns])

In [57]:
drop_date_added = removewhitespace.drop("date_added")

In [58]:
df_cleaned = drop_date_added.withColumn("listed_in", regexp_replace(col("listed_in"), "[&-]", ","))

In [59]:
explode_listed_in= df_cleaned.withColumn("Category", explode(split("listed_in", ","))).drop("listed_in")

In [60]:
df_exploded = explode_listed_in.withColumn("Category", trim(col("Category")))

In [61]:
df_cleaned_years = df_exploded.filter(~col("Category").rlike("^[0-9]{4}$"))  # Remove years
df_cleaned_duration = df_cleaned_years.filter(~col("Category").rlike("^[0-9]+ min$")&
    ~col("Category").rlike("Seasons"))  # Remove durations

In [62]:
country_list = ["Germany", "France", "United States", "India","Nigeria"]

# Define a UDF to check if a value is a country
def is_country(value):
    if value in country_list:
        return value
    return None

In [63]:
from pyspark.sql.functions import udf
is_country_udf = udf(is_country, StringType())

In [64]:
df_with_countries = df_cleaned_duration.withColumn("country_detected", is_country_udf(col("Category")))

In [65]:
df_without_countries = df_with_countries.filter(col("country_detected").isNull()).drop("country_detected")

In [66]:
category_mapping = {
    'TV Shows': 'Television',
    'International': 'Television',
    'Music Videos and Concerts': 'Music',
    'Romance': 'Romance',
    'Young Adult Audience': 'Young Adult Audience',
    'Danny Tellez': 'Other',
    'Yakima Canutt': 'Other',
    'Entertainment': 'Entertainment',
    'Guinn ``big Boy'' Williams': 'Other',
    'Adventure': 'Adventure',
    'Sports': 'Sports',
    'Faith and Spirituality': 'Spiritual',
    'Drama': 'Drama',
    'Military and War': 'Military and War',
    'Documentary': 'Documentary',
    'Manuel González Sabin Cañita': 'Other',
    'Fitness': 'Health',
    'Fantasy': 'Fantasy',
    'LGBTQ': 'LGBTQ',
    'Tom Lingham': 'Other',
    'and Culture': 'Culture',
    'BJ Minor': 'Other',
    'Mark Salidino': 'Other',
    'Lew Luana': 'Other',
    'Martin Kove': 'Other',
    'Iron Eyes Cody': 'Other',
    'Crystal Howell': 'Other',
    'Ivan Miller': 'Other',
    'Tex Palmer': 'Other',
    'Anime': 'Animation',
    'Suspense': 'Thriller',
    'Charles Arnt': 'Other',
    'Animation': 'Animation',
    'Arts': 'Art',
    'Kayden Bryce': 'Other',
    'Special Interest': 'Special Interest',
    'Mic Larry': 'Other',
    'Kids': 'Children',
    'Science Fiction': 'Sci-Fi',
    'Tony Lee': 'Other',
    'Arthouse': 'Art',
    'Horror': 'Horror',
    'George Cleveland': 'Other',
    'Western': 'Western',
    'Luxembourg': 'Other',
    'Reese AKA LowKeyRG': 'Other',
    'William Haade': 'Other',
    'Comedy': 'Comedy',
    'Action': 'Action',
    'Historical': 'History',
    'Talk Show and Variety': 'Talk Show',
    'Glenn Strange': 'Other',
    'Unscripted': 'Reality',
    'October 16': 'Other',
    'Romantic TV Shows': 'Romance',
    'Korean TV Shows': 'Television',
    'Patricia López Arnaiz': 'Other',
    'Comedies': 'Comedy',
    'Science': 'Science',
    'Faith': 'Spiritual',
    'Musicals': 'Musical',
    'Anime Features': 'Animation',
    'Marta Larralde': 'Other',
    'Sports Movies': 'Sports',
    'Reality TV': 'Reality',
    'April 5': 'Other',
    'TV Horror': 'Horror',
    'Janeane Garofalo': 'Other',
    'Crime TV Shows': 'Crime',
    'Margaret Cho': 'Other',
    'Cult TV': 'Cult',
    'Independent Movies': 'Indie',
    'Language TV Shows': 'Television',
    'Talk Shows': 'Talk Show',
    'Anime Series': 'Animation',
    "Kids' TV": 'Children',
    'Dramas': 'Drama',
    '2 Seasons': 'Other',
    'Eden Marryshow': 'Other',
    'Romantic Movies': 'Romance',
    'TV Mysteries': 'Mystery',
    'Spanish': 'Other',
    'British TV Shows': 'Television',
    'Teen TV Shows': 'Television',
    'Akin Lewis': 'Other',
    'International Movies': 'Movies',
    'TV Action': 'Action',
    'Movies': 'Movies',
    'TV Comedies': 'Comedy',
    'Horror Movies': 'Horror',
    'Thrillers': 'Thriller',
    'TV Dramas': 'Drama',
    'Heavy D': 'Other',
    'Music': 'Music',
    'Spirituality': 'Spiritual',
    'Family Movies': 'Family',
    'TV Sci': 'Sci-Fi',
    'International TV Shows': 'Television',
    'Up Comedy': 'Comedy',
    'Docuseries': 'Documentary',
    'Classic Movies': 'Classic',
    'TV Thrillers': 'Thriller',
    'Nature TV': 'Nature',
    'January 16': 'Other',
    'Netflix': 'Other',
    'Children': 'Children',
    'Stand': 'Comedy',
    'LGBTQ Movies': 'LGBTQ',
    'United Kingdom': 'Other',
    'Cult Movies': 'Cult',
    'Classic': 'Classic',
    'Wellness': 'Health',
    'Crime': 'Crime',
    'Thriller': 'Thriller',
    'Food': 'Food',
    'Teen': 'Teen',
    'Sketch Comedy': 'Comedy',
    'Health': 'Health',
    'Culture': 'Culture',
    'Latino': 'Other',
    'Family': 'Family',
    'Reality': 'Reality',
    'Sitcom': 'Comedy',
    'History': 'History',
    'Mystery': 'Mystery',
    'Classics': 'Classic',
    'Stand Up': 'Comedy',
    'Technology': 'Technology',
    'Lifestyle': 'Lifestyle',
    'Game Shows': 'Game Show',
    'Black Stories': 'Black Stories',
    'Cartoons': 'Animation',
    'Cooking': 'Food',
    'LGBTQ+': 'LGBTQ',
    'Adult Animation': 'Animation',
    'Late Night': 'Talk Show',
    'News': 'News',
    'Kosovo': 'Other',
    'Buddy': 'Other',
    'Police/Cop': 'Crime',
    'Survival': 'Survival',
    'Travel': 'Travel',
    'Biographical': 'Biography',
    'Spy/Espionage': 'Thriller',
    'Soap Opera / Melodrama': 'Drama',
    'Disaster': 'Disaster',
    'Series': 'Series',
    'Musical': 'Musical',
    'Concert Film': 'Music',
    'Romantic Comedy': 'Romance Comedy',
    'Variety': 'Variety',
    'Nature': 'Nature',
    'Coming of Age': 'Movies',
    'Animals': 'Animals',
    'Superhero': 'Action',
    'Anthology': 'Movies',
    'Medical': 'Medical',
    'Dance': 'Screen Dance',
    'Parody': 'Comedy',
    'Talk Show': 'Talk Show',
    'Game Show / Competition': 'Game Show',
    'Cameron Fraser': 'Other'
}

In [67]:
standardized_categories_df = df_without_countries.withColumn(
    "Standardized_Category",
    F.when(
        F.col("Category").isin(list(category_mapping.keys())),
        F.col("Category")
    ).otherwise("Other")
)

In [68]:
def map_categories(category):
    return category_mapping.get(category, 'Other')

In [69]:
map_categories_udf = F.udf(map_categories)

In [70]:
standardized_categories_df = standardized_categories_df.withColumn(
    "Standardized_Category",
    map_categories_udf(F.col("Standardized_Category"))
)

In [71]:
drop_category_df = standardized_categories_df.drop("Category")

In [72]:
drop_category_df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: string (nullable = true)
 |-- user_reviews: string (nullable = true)
 |-- user_rating: string (nullable = true)
 |-- Standardized_Category: string (nullable = true)



In [73]:
df_casted = drop_category_df.withColumn("total_views", col("total_views").cast(IntegerType())) \
              .withColumn("user_reviews", col("user_reviews").cast(IntegerType())) \
              .withColumn("user_rating", col("user_rating").cast(FloatType()))

In [74]:
director_nulls= df_casted.replace('Null', "unknown")

In [75]:
cast_nulls= director_nulls.replace('Null', "unknown")

In [76]:
# Filter out rows where 'director' and 'cast' columns that are fully numeric (integers)
numeric_pattern = "^[0-9]+$"

filtered_df = cast_nulls.filter(
    ~(
        col("director").rlike(numeric_pattern) &
        col("cast").rlike(numeric_pattern)
    )
)

In [77]:
filtered_df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- user_rating: float (nullable = true)
 |-- Standardized_Category: string (nullable = true)



In [78]:
description_nulls= filtered_df.replace('Null', "unknown")

In [79]:
casting_year = description_nulls.withColumn("release_year", col("release_year").cast(IntegerType()))

In [80]:
casting_year.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- user_rating: float (nullable = true)
 |-- Standardized_Category: string (nullable = true)



In [81]:
# Define thresholds for classification
short_threshold = 90   # minutes
medium_threshold = 150 # minutes

In [82]:
# Classify movies based on duration directly
bins_movies = casting_year.withColumn(
    "movie_duration_category",
    when(regexp_extract(col("duration"), r"(\d+)", 1).cast("integer") <= short_threshold, "short")
    .when(
        (regexp_extract(col("duration"), r"(\d+)", 1).cast("integer") > short_threshold) &
        (regexp_extract(col("duration"), r"(\d+)", 1).cast("integer") <= medium_threshold),
        "medium"
    )
    .otherwise("long")
)

In [83]:
bins_movies.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- user_rating: float (nullable = true)
 |-- Standardized_Category: string (nullable = true)
 |-- movie_duration_category: string (nullable = false)



In [84]:
country_nulls= bins_movies.replace('Null', "unknown")

In [85]:
duration_nulls= country_nulls.replace('Null', "unknown")

In [86]:
duration_nulls.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_views: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- user_rating: float (nullable = true)
 |-- Standardized_Category: string (nullable = true)
 |-- movie_duration_category: string (nullable = false)



In [87]:
exploded_countries = duration_nulls.withColumn("country", explode(split("country", ",")))
dropnullcountries = exploded_countries.na.drop(subset=["country"])

In [88]:
checkpoint_dir = "/mnt/f/MoviesProjectCheckpoint"

def write_to_sql_server(batch_df, batch_id):
    try:
        batch_df.write \
            .format("jdbc") \
            .option("url", "jdbc:sqlserver://192.168.1.11:1433;databaseName=MoviesProject;user=mostafa;password=mostafa7amdy;") \
            .option("dbtable", "moviesProject") \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .option("checkpointLocation", checkpoint_dir) \
            .mode("append")\
            .save()
    except Exception as e:
        print(str(e))

# Start the stream and write to SQL Server
dropnullcountries.writeStream \
    .foreachBatch(write_to_sql_server) \
    .start() 


24/09/23 18:25:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/09/23 18:25:42 INFO ResolveWriteToStream: Checkpoint root file:///tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3 resolved to file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3.
24/09/23 18:25:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/09/23 18:25:42 INFO CheckpointFileManager: Writing atomically to file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/metadata using temp file file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/.metadata.5ee63f27-0888-41a3-b4ac-12443b20cbd4.tmp
24/09/23 18:25:42 INFO 

24/09/23 18:25:43 INFO OffsetSeqLog: BatchIds found from listing: 
24/09/23 18:25:43 INFO OffsetSeqLog: BatchIds found from listing: 
24/09/23 18:25:43 INFO MicroBatchExecution: Starting new streaming query.
24/09/23 18:25:43 INFO MicroBatchExecution: Stream started from {}


In [89]:
duration_nulls.isStreaming

24/09/23 18:25:43 INFO AdminClientConfig: AdminClientConfig values: 
	bootstrap.servers = [localhost:9092]
	client.dns.lookup = use_all_dns_ips
	client.id = 
	connections.max.idle.ms = 300000
	default.api.timeout.ms = 60000
	metadata.max.age.ms = 300000
	metric.reporters = []
	metrics.num.samples = 2
	metrics.recording.level = INFO
	metrics.sample.window.ms = 30000
	receive.buffer.bytes = 65536
	reconnect.backoff.max.ms = 1000
	reconnect.backoff.ms = 50
	request.timeout.ms = 30000
	retries = 2147483647
	retry.backoff.ms = 100
	sasl.client.callback.handler.class = null
	sasl.jaas.config = null
	sasl.kerberos.kinit.cmd = /usr/bin/kinit
	sasl.kerberos.min.time.before.relogin = 60000
	sasl.kerberos.service.name = null
	sasl.kerberos.ticket.renew.jitter = 0.05
	sasl.kerberos.ticket.renew.window.factor = 0.8
	sasl.login.callback.handler.class = null
	sasl.login.class = null
	sasl.login.refresh.buffer.seconds = 300
	sasl.login.refresh.min.period.seconds = 60
	sasl.login.refresh.window.factor 

True

24/09/23 18:25:43 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/09/23 18:25:43 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/09/23 18:25:43 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/09/23 18:25:43 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/09/23 18:25:43 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
24/09/23 18:25:43 INFO AppInfoParser: Kafka version: 2.8.1
24/09/23 18:25:43 INFO AppInfoParser: Kafka commitId: 839b886f9b732b15
24/09/23 18:25:43 INFO AppInfoParser: Kafka startTimeMs: 1727105143099


24/09/23 18:25:43 INFO CheckpointFileManager: Writing atomically to file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/sources/0/0 using temp file file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/sources/0/.0.406669b3-533a-4909-9272-6d72471d25da.tmp
24/09/23 18:25:43 INFO CheckpointFileManager: Renamed temp file file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/sources/0/.0.406669b3-533a-4909-9272-6d72471d25da.tmp to file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/sources/0/0
24/09/23 18:25:43 INFO KafkaMicroBatchStream: Initial offsets: {"moviesProject":{"2":0,"1":0,"3":0,"0":0}}
24/09/23 18:25:43 INFO CheckpointFileManager: Writing atomically to file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/offsets/0 using temp file file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/offsets/.0.cfcf501c-e326-4acf-8781-978221374d24.tmp
24/09/23 18:25:43 INFO CheckpointFileManager: Renamed temp file file:/tmp/temporary-f0eb9c26-3b0a-430d-9d57-1547ac0de9b3/o